In [3]:
import numpy as np
from scipy.stats import norm, mvn
from scipy.optimize import fsolve

# Function to compute the bivariate normal CDF M(a, b; rho)
def M_bivariate_normal(a, b, rho):
    lower = [-np.inf, -np.inf]
    upper = [a, b]
    mean = [0, 0]
    cov_matrix = [[1, rho], [rho, 1]]  # Covariance matrix with correlation rho
    p, _ = mvn.mvnun(lower, upper, mean, cov_matrix)
    return p

# Black-Scholes formula for European call option price
def black_scholes_call(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price

# Roll-Geske-Whaley formula for American call option
def roll_geske_whaley(S0, K, r, T, sigma, D1, t1):
    # a1 and a2
    a1 = (np.log((S0 - D1 * np.exp(-r * t1)) / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    a2 = a1 - sigma * np.sqrt(T)

    # Solving for S_star
    def bs_condition(S_star):
        return black_scholes_call(S_star, K, T - t1, r, sigma) - (S_star + D1 - K)

    S_star = fsolve(bs_condition, S0)[0]

    # b1 and b2
    b1 = (np.log((S0 - D1 * np.exp(-r * t1)) / S_star) + (r + 0.5 * sigma**2) * t1) / (sigma * np.sqrt(t1))
    b2 = b1 - sigma * np.sqrt(t1)

    # Correlation between the two variables
    rho = -np.sqrt(t1 / T)

    # Roll-Geske-Whaley formula components using bivariate normal CDF M(a, b; rho)
    M1 = M_bivariate_normal(a1, -b1, rho)
    M2 = M_bivariate_normal(a2, -b2, rho)

    # Final price calculation
    call_price = (S0 - D1 * np.exp(-r * t1)) * M1 + (S0 - D1 * np.exp(-r * t1)) * norm.cdf(b1)
    call_price -= K * np.exp(-r * T) * M2 + (K - D1) * np.exp(-r * T) * norm.cdf(b2)

    return call_price

# Parameters
S0 = 100  # Current stock price
K = 100   # Strike price
r = 0.05  # Risk-free interest rate
T = 1.0   # Time to maturity (years)
sigma = 0.2  # Volatility
D1 = 2.0  # Dividend payment
t1 = 0.5  # Time of dividend payment (years)

# Calculate the American call option price
call_price = roll_geske_whaley(S0, K, r, T, sigma, D1, t1)
print(f"Roll-Geske-Whaley American Call Option Price: {call_price:.4f}")


Roll-Geske-Whaley American Call Option Price: 9.2447


<ipython-input-3-bfa89bdc4fa4>:31: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  S_star = fsolve(bs_condition, S0)[0]
<ipython-input-3-bfa89bdc4fa4>:11: DeprecationWarning: `scipy.stats.mvn.mvnun` is deprecated along with the `scipy.stats.mvn` namespace. `scipy.stats.mvn.mvnun` will be removed in SciPy 1.14.0, and the `scipy.stats.mvn` namespace will be removed in SciPy 2.0.0.
  p, _ = mvn.mvnun(lower, upper, mean, cov_matrix)
